# Obtaining data through the ENTSO-E API

In [ ]:
import pandas as pd
from entsoe import EntsoePandasClient

path = '/Users/jakagodec/Desktop/ist/es/project3/EnergyPrices/'

## Token
Each user can access their own token by asking the ENTSO-E Transparency Platform support team for access (__[instructions](https://amsleser.no/blog/post/21-obtaining-api-token-from-entso-e)__). I did not want to publish my token so this is the solution for now.

In [ ]:
with open(path + 'data/token.txt', 'r') as t:
    token = t.read()
print('Token sucessfully loaded: ', token)

client = EntsoePandasClient(api_key=token)

## Testing API calls
More detailed description can be found on the `entsoe-py` __[github](https://github.com/EnergieID/entsoe-py#EntsoePandasClient)__. I tested some stuff for the countries we are interested in (broadly said the Balkans, although data for some countries - Bosnia and Herzegovina, Montenegro and North Macedonia - is missing). Country codes:
* Slovenia: `SI`
* Serbia: `RS`
* Croatia: `HR`
* Bulgaria: `BG`
* Romania: `RO`
* Greece: `GR`

In [ ]:
country = 'SI'
start = pd.Timestamp('20220101', tz='UTC')
end = pd.Timestamp('20230101', tz='UTC')

In [ ]:
prices = client.query_day_ahead_prices(country_code='BG', start=start, end=end)
prices

In [ ]:
generation = client.query_generation(country_code='SI', start=start, end=end)
generation

In [ ]:
generation.to_csv(path + 'data/SI_raw/SI_generation.csv')

In [ ]:
unavailability = client.query_unavailability_of_generation_units(country_code='RS', start=start, end=end)
unavailability

In [ ]:
imports = client.query_import(country_code='RS', start=start, end=end)
imports

## Raw data
Query desired features and save raw files as CSV.

In [ ]:
def get_country_entsoe_raw(country_code, start_ts, end_ts):
    """
    Query all data and save the outputs to CSV files.
    """
    print('Obtaining data for country: ' + country_code)
    country_path = path + 'data/' + country_code + '_raw/'
    
    # save prices
    prices = client.query_day_ahead_prices(country_code=country_code, start=start_ts, end=end_ts)
    prices.to_csv(country_path + country_code + '_prices.csv')
    print('====== Saved prices data ============', end='\r')

    # save generation
    generation = client.query_generation(country_code=country_code, start=start_ts, end=end_ts)
    generation.to_csv(country_path + country_code + '_generation.csv')
    print('====== Saved generation data ========', end='\r')

    # save consumption (actual and forecasted)
    load = client.query_load_and_forecast(country_code=country_code, start=start_ts, end=end_ts)
    load.to_csv(country_path + country_code + '_load.csv')
    print('====== Saved load data ==============', end='\r')

    # save unavailability: HR has none
    if country_code.lower() != 'HR':
        unav = client.query_unavailability_of_generation_units(country_code=country_code, start=start_ts, end=end_ts)
        unav.to_csv(country_path + country_code + '_unavailability.csv')
        print('====== Saved unavailability data ====', end='\r')
    
    # save imports
    imprt = client.query_import(country_code=country_code, start=start_ts, end=end_ts)
    imprt.to_csv(country_path + country_code + '_import.csv')
    print('===> ' + country_code + ' done =======================')


In [ ]:
t_start = pd.Timestamp('20220101', tz='UTC')
t_end = pd.Timestamp('20230301', tz='UTC')

get_country_entsoe_raw('SI', t_start, t_end)

In [ ]:
codes = ['RS', 'RO', 'HR', 'GR', 'BG']

for c in codes:
    get_country_entsoe_raw(c, t_start, t_end)

## Weather data
I manually saved the data from __[open-meteo.com](https://open-meteo.com/en/docs/historical-weather-api#latitude=44.43&longitude=26.11&start_date=2022-01-01&end_date=2023-03-01&hourly=temperature_2m,precipitation,direct_radiation&timezone=Europe%2FBerlin)__ for the capital cities of our countries, with the UTC timezone (same as the ENTSO-E data).